# Test Migration Plan

Test Migration plan class (in classes folder) by:
- First make ILP problem identical to last time
- Make mig class based on users created
- Try to devise a way to take ILP decision values and make plan out of that

In [21]:
# Import Generic Classes
import numpy as np
import copy

# Import All Custom Classes
import os, sys
sys.path.append(os.path.pardir+"/classes")

from Server import *
from User import *
from Link import *
from Job import *

# Import Solver Classes
from Optim_PlanGenerator import *
from SeqGreedy_PlanGenerator import*

In [269]:
import numpy as np
import math

class Migration_Plans:
    """
    Migration_Plans: 
        - Collects all migration plans generated for an entire system
        - mig_plan_dict : 6 x time_steps np array with rows
            - timeslot, user_active_flag, usr_voronoi, source_svr, dest_svr, mig_rate
    """
    
    def __init__(self, users, jobs, sim_params):
        """
        users - list of user class (used to extract user location)
        time_steps - how many timesteps to simulate user movement for
        """
        
        self.mig_plan_dict = {}
        self.sim_params = sim_params
        
        # Initialize dictionary 
        self.dict_init(users,jobs,sim_params.time_steps)
    
    """
    Init Helper Function
    """
    
    def dict_init(self, users, jobs, time_steps):
        
        param_collection = ["time_slot", "user_active_flag", 
                            "user_voronoi", "source_server", 
                            "dest_server", "mig_rate",
                            "mig_link_id", "service_link_id",
                            "service_thruput", "latency"]
        
        for u in range(len(users)):
            temp_usr_dict = {}
            
            for p in param_collection:
                temp_usr_dict[p] = np.zeros(time_steps)
            
            # Record active time and user voronoi
            for t in range(int(time_steps)):
                temp_usr_dict["time_slot"][t] = t
                temp_usr_dict["user_voronoi"][t] = users[u].user_voronoi_true[t]
                temp_usr_dict["user_active_flag"][t] = jobs[u].active_time[t]
            
            self.mig_plan_dict[u] = temp_usr_dict
    
    """
    Extraction Functions
    """
    
    def from_ILP(self, ILP_prob, solve_flag = True):
        """
        From decision variables h we want to replace the zero vectors of 
        self.mig_plan_dict with relevant values based on the decision vars
        
        Input: ILP_prob - an Optim_PlanGenerator object that already has been optimized
        """
        
        if solve_flag:
            ILP_prob.prob.solve()
        
        # 1. Loop through all h_vars and obtain those that have been selected
        h_hit_keys = []
        
        for h_key in ILP_prob.h_vars.keys():
            if ILP_prob.h_vars[h_key].varValue>0:
                h_hit_keys += [h_key]
            
        
        # 2. Loop through each of the users and isolate variables that pertain to them
        for j in range(len(ILP_prob.jobs)):
            uh_keys = []
            
            # Collect all keys from a certain user
            for hit_key in h_hit_keys:
                if hit_key[0] == j: # If of the correct user
                    uh_keys += [hit_key]
            
            
            # Reorder the hit keys in terms of time
            uh_keys_ordered = []
            curr_time = -1 
            for l in range(len(uh_keys)):
                time_key = None
                for key in uh_keys:
                    if key[3] == curr_time:
                        break
                uh_keys_ordered += [key]
                curr_time = key[4]
                uh_keys.remove(key)
        
            self.ILP_plan_extract(uh_keys_ordered,j)
        
        # 3. Reserve Resources From Resource Constraints
        for j in range(len(ILP_prob.jobs)):
            placement_rsrc = ILP_prob.jobs[j].placement_rsrc
            mig_rsrc = ILP_prob.jobs[j].migration_rsrc
            service_bw = ILP_prob.jobs[j].thruput_req
            
            plan_dict = self.mig_plan_dict[j]
            for t in range(self.sim_params.time_steps):
                # Reserve Placement Cost & Mig link cost
                source_svr = int(plan_dict['source_server'][t])
                dest_svr = int(plan_dict['dest_server'][t])
                mig_rate = plan_dict['mig_rate'][t]
                path_idx = int(plan_dict['mig_link_id'][t])
                
                if source_svr == dest_svr:
                    ILP_prob.resource_constraints.server_rsrc[source_svr,:,t] -= placement_rsrc
                else:
                    ILP_prob.resource_constraints.server_rsrc[source_svr,:,t] -= placement_rsrc
                    ILP_prob.resource_constraints.server_rsrc[dest_svr,:,t] -= placement_rsrc
                    
                    avail_link = ILP_prob.resource_constraints.link_rsrc[:,:,t]
                    mig_links = ILP_prob.links.get_subpath(source_svr,dest_svr,path_idx)
                    remain_link = avail_link - (mig_rsrc*mig_rate*mig_links)
                    
                    ILP_prob.resource_constraints.link_rsrc[:,:,t] = remain_link
                
                # Reserve Expected Service BAndwidth Cost
                avail_link = ILP_prob.resource_constraints.link_rsrc[:,:,t]
                exp_service = np.zeros((len(ILP_prob.servers),len(ILP_prob.servers)))
                
                for s_var in range(len(ILP_prob.servers)):
                    if s_var != source_svr:
                        avg_link = ILP_prob.links.get_avgpath(source_svr,s_var)
                        usr_job_flag = ILP_prob.users[j].server_prob[s_var,t]
                        expected_sbw = np.multiply(service_bw, avg_link)
                        exp_service += expected_sbw
                
                remain_link = avail_link - exp_service
                ILP_prob.resource_constraints.link_rsrc[:,:,t] = remain_link
                
        
        self.prob = ILP_prob
        self.service_path_selection()
        self.thruput_selection()
        return
    
    def from_seq_greedy(self,SG_prob):
        """
        From the migration plan problem, do the entire procedure of reserving resources and 
        generating final migration plan -- how will we do this for batch?
        """
        
        # Should add convert node informations

        self.prob = SG_prob
        
        # Loop through time and user to generate incoming plans
        for t in range(self.sim_params.time_steps):
            for j in range(len(self.prob.jobs)):
                
                # 1. Check for user arrival time
                if t == self.prob.jobs[j].arrival_time:
                    
                    # 0. Update user server prob
                    self.prob.users[j].update_voronoi_probs(time_passed = t)
                    
                    node_bans = []
                    path_bans = []
                    approved_flag = False
                    
                    while_idx = 0
                    
                    while not approved_flag:
                        print("usr:",j,"reserve:",while_idx)
                        while_idx += 1
                    
                        # 2. If user arrives, make plan
                        self.prob.calc_all_costs(j=0)
                        self.prob.obtain_minimum_cost_j(j=0)

                        # Start Node and End Node of Mig plan
                        start_node = self.prob.convert_st2node[j][(-1,-1)]
                        end_node = self.prob.convert_st2node[j][(-1,self.sim_params.time_steps)]

                        node_num, link_num = self.prob.dijkstra_j(j=j,start_node=start_node,
                                                                     end_node=end_node)
                        
                        
                        # 3. Repeat resource reservation until no conflicts --> or reject job
                        node_bans, path_bans, approved_flag = self.prob.check_reserve_resource(j,
                                                              node_num,link_num)
                        
                        # Update cost graph
                        if not approved_flag:
                            self.prob.update_costs(j, node_bans,path_bans)
                    
                    
                    # Extract plan and record to system
                    self.seq_greedy_plan_extract(node_orders=node_num, 
                                                 link_path_orders=link_num, 
                                                 job_num=j)

        self.service_path_selection()
        self.thruput_selection()
    
    """
    Extraction function helpers
    """
    
    # ILP
    
    def ILP_plan_extract(self, uh_keys_ordered, job_num):
        """
        Loop through the ordered selected keys for a single user 
        And generate np arrays that will correspond to plans
        Inputs:
            uh_keys_ordered: list of tupels that represent h-variables in ILP Solution
            job_num: job id 
        """
        
        active = True
        inactive = False
        
        # Loop through each of the keys (Use switch cases below)
        for uh_key in uh_keys_ordered:
            start_time, end_time = uh_key[3], uh_key[4]
            source_server,dest_server = uh_key[1], uh_key[2]
            link_path = uh_key[5]
            
            case = (source_server >= 0, dest_server >= 0) # server source, dest active/inactive
            
            if case == (active, active) or case == (inactive,inactive):
                self.mig_plan_dict[job_num]["source_server"][start_time:end_time] = source_server
                self.mig_plan_dict[job_num]["dest_server"][start_time:end_time] = dest_server
                
                # Migration length find
                if source_server != dest_server:
                    mig_length = end_time - start_time
                    self.mig_plan_dict[job_num]["mig_rate"][start_time:end_time] = 1/mig_length
                    self.mig_plan_dict[job_num]["mig_link_id"][start_time:end_time] = link_path
                    
            elif case == (inactive, active) or case == (active, inactive):
                # The entire column in the plan is considered inactive/active
                self.mig_plan_dict[job_num]["source_server"][start_time:end_time] = source_server
                self.mig_plan_dict[job_num]["dest_server"][start_time:end_time] = source_server
    
    # HEuristic
    
    def seq_greedy_plan_extract(self, node_orders, link_path_orders, job_num):
        """
        Loop through the ordered selected nodes for a single user 
        And generate np arrays that will correspond to plans
        Inputs:
            node_orders: Sequence of nodes in mig graph that make up a plan
            link_path_orders : which path was taken between two nodes in mig graph
            job_num: job id 
        """
        
        active = True
        inactive = False
        
        # Pull pairs of nodes that are connected together
        pair_list = []
        path_idx = 0
        for i in range(len(node_orders)-1):
            pair_list += [(node_orders[i],node_orders[i+1])]
        
        # Loop through each of the keys (Use switch cases below)
        for (node1,node2) in pair_list:
            (source_server, start_time) = self.prob.convert_node2st[job_num][node1]
            (dest_server, end_time) = self.prob.convert_node2st[job_num][node2]
            link_path = link_path_orders[path_idx]
            path_idx += 1
            
            case = (source_server >= 0, dest_server >= 0) # server source, dest active/inactive
            
            if case == (active, active) or case == (inactive,inactive):
                self.mig_plan_dict[job_num]["source_server"][start_time:end_time] = source_server
                self.mig_plan_dict[job_num]["dest_server"][start_time:end_time] = dest_server
                
                # Migration length find
                if source_server != dest_server:
                    mig_length = end_time - start_time
                    self.mig_plan_dict[job_num]["mig_rate"][start_time:end_time] = 1/mig_length
                    self.mig_plan_dict[job_num]["mig_link_id"][start_time:end_time] = link_path
                    
            elif case == (inactive, active) or case == (active, inactive):
                # The entire column in the plan is considered inactive/active
                self.mig_plan_dict[job_num]["source_server"][start_time:end_time] = source_server
                self.mig_plan_dict[job_num]["dest_server"][start_time:end_time] = source_server
    
        
    def service_path_selection(self):
        """
        take into a consideration the resources at each link at each timestep, and determine
        Inputs:
        links - a link instance of the simulation
        jobs - a list of job objects each with their job size 
        
        Updates migration plan to determine throughput of service at each instance
        """
        
        switch_latency = self.prob.links.switch_delay
        dist_latency = self.prob.links.dist_delay
        server_distances = self.prob.links.calc_distance(self.prob.servers)
        
        # Loop thru plan - pick service and calc latency for each ts
        for j,t in itertools.product(range(len(self.prob.jobs)),range(self.sim_params.time_steps)):
            usr_svr = int(self.mig_plan_dict[j]["user_voronoi"][t])
            job_svr = int(self.mig_plan_dict[j]["source_server"][t])
            
            if usr_svr != job_svr:
                # Calculate which path
                num_path = int(self.prob.links.num_path[job_svr,usr_svr])
                select_path = np.random.randint(0,num_path)
                self.mig_plan_dict[j]['service_link_id'][t] = select_path
            
                # Calculate Latency
                service_path = self.prob.links.get_subpath(job_svr,usr_svr,select_path)
                num_switch = np.sum(np.sum(service_path,axis=1),axis=0)
                
                latency_dists = np.multiply(service_path,server_distances)
                num_dist = np.sum(np.sum(latency_dists,axis=1),axis=0)
               
                self.mig_plan_dict[j]['latency'][t] = switch_latency * num_switch + num_dist * dist_latency
            
            else:
                self.mig_plan_dict[j]["service_link_id"][t] = -1
                
    def thruput_selection(self):
        """
        After running service_path_selection() we can pick thruput of each job at each timestep
        """
        
        # Loop through each timestep 
        for t in range(self.sim_params.time_steps):
            
            service_thruput = np.zeros(self.prob.links.distances.shape)
            job_thruputs = []
            
            # Add all 
            for j in range(len(self.prob.jobs)):
                # Add to list job idx and thruput
                job_thruputs += [self.prob.jobs[j].thruput_req]
            
            approved_flag = False
            
            # Adjust throughputs for this timestep
            while not approved_flag:
                for j in range(len(self.prob.jobs)):
                    if self.mig_plan_dict[j]["service_link_id"][t] > -1:
                        usr_svr = int(self.mig_plan_dict[j]["user_voronoi"][t])
                        job_svr = int(self.mig_plan_dict[j]["source_server"][t])
                        path_id = int(self.mig_plan_dict[j]["service_link_id"][t])

                        service_links_j = self.prob.links.get_subpath(job_svr,usr_svr,path_id)
                        service_thruput += job_thruputs[j] * service_links_j

                remainder_link = self.prob.resource_constraints.link_rsrc[:,:,t] - service_thruput
                
                one_coor = zip(*np.where(remainder_link < 0))

                if len(list(one_coor)) == 0:
                    approved_flag = True
                    continue
                    
                struck_jobs = []
                
                for (s1,s2) in one_coor:
                    for j in range(len(self.prob.jobs)):
                        if self.mig_plan_dict[j]["service_link_id"][t] > -1:
                            usr_svr = int(self.mig_plan_dict[j]["user_voronoi"][t])
                            job_svr = int(self.mig_plan_dict[j]["source_server"][t])
                            path_id = int(self.mig_plan_dict[j]["service_link_id"][t])

                            service_links_j = self.prob.links.get_subpath(job_svr,usr_svr,path_id)

                            if service_links_j[s1,s2] > 0 and (j not in struck_jobs):
                                job_thruputs[j] *= 0.9
                                struck_jobs += [j]
                                
            # Record throughput for each job
            for j in range(len(self.prob.jobs)):
                thru_flag = (self.mig_plan_dict[j]["service_link_id"][t] > -1)
                self.mig_plan_dict[j]["service_thruput"][t] = thru_flag * job_thruputs[j]

### Make Users, Servers, Jobs

In [276]:
"""
Make Simulation Parameters
"""
sim_param = Sim_Params(time_steps=5, x_length = 5, y_length = 5, max_edge_length=2)
boundaries = np.array([[0,sim_param.x_length],[0,sim_param.y_length]])


"""
Make Job Profiles
"""
# REsources used are CPU (no. cores) storage (GB), and RAM (GB)
# througput is in mb/s
# Latency is in ms

job_profile1 = Job_Profile(job_name = "VR",
                           latency_req_range=[0, 0], 
                           thruput_req_range=[50/1000, 200/1000], 
                           length_range=[5,5],  
                           placement_rsrc_range = np.array([[2,3],[8,16],[2,5]]),
                           migration_amt_range = [5, 10],
                           latency_penalty_range = [1,11])#[0.05, 0.1]) 

job_profile2 = Job_Profile(job_name = "Assistant",
                           latency_req_range=[100, 200],
                           thruput_req_range=[5/1000, 20/1000],
                           length_range=[1,5],
                           placement_rsrc_range = np.array([[1,1],[0.5,1],[0.5,1]]),
                           migration_amt_range = [0.5, 1],
                           latency_penalty_range = [0.01, 0.05])

job_profile3 = Job_Profile(job_name = "AR",
                           latency_req_range=[50, 80], 
                           thruput_req_range=[20/1000, 50/1000],
                           length_range=[1,5],
                           placement_rsrc_range = np.array([[1,2],[2,4],[1,2]]),
                           migration_amt_range = [2, 3],
                           latency_penalty_range = [0.03, 0.08])

job_profiles = [job_profile1, job_profile2, job_profile3]


"""
Make Servers
"""

# Server Settings
num_server_l1 = 3
num_server_l2 = 2
num_server_l3 = 1

num_resource = 3
weak_range = np.array([[4,8],[1000,1500],[4,16]])
strong_range = np.array([[50,100],[100000,150000],[300,600]])

rsrc_cost = np.array([0.03, 0.01, 0.05])

# Generate Server
servers_l1 = []
servers_l2 = []
servers_l3 = []
idx_counter = 0

for i in range(num_server_l1):
    servers_l1.append(Server(boundaries,level=1,rand_locs=True,locs=None))
    servers_l1[-1].server_resources(num_resource, weak_range, strong_range)
    servers_l1[-1].assign_id(idx_counter)
    servers_l1[-1].server_resources_cost(num_resource,rsrc_cost)
    idx_counter += 1
    
for i in range(num_server_l2):
    servers_l2.append(Server(boundaries,level=2,rand_locs=True,locs=None))
    servers_l2[-1].server_resources(num_resource, weak_range, strong_range)
    servers_l2[-1].assign_id(idx_counter)
    servers_l2[-1].server_resources_cost(num_resource,rsrc_cost)
    idx_counter += 1
    
for i in range(num_server_l3):
    servers_l3.append(Server(boundaries,level=3,rand_locs=False,locs=np.array([200,200])))
    servers_l3[-1].server_resources(num_resource, weak_range, strong_range)
    servers_l3[-1].assign_id(idx_counter)
    servers_l3[-1].server_resources_cost(num_resource,rsrc_cost)
    idx_counter += 1
    
servers = servers_l1 + servers_l2 + servers_l3


"""
Make Links
"""

# Link Settings
num_link = [0,1,2]
prob_link = [0,1,0]
lv_minmax = np.array(([[500,1000],[10000,20000],[30000,50000]]))
lv1_transmission = 1
link_costs = np.array([0.05, 0.02, 0.01])
latency_settings = [1000, 100] #[ms per switch, ms per mile]

links = Link(servers, num_link, prob_link, lv_minmax, link_costs, latency_settings,lv1_transmission)


"""
Make Users
"""

# User Settings
num_user_m0 = 0 # Pedestrian
num_user_m1 = 0 # Public Transport
num_user_m2 = 1 # Vehicle

max_speed = 2.5
lamdas = [1/0.25,1/0.83,1/1.67] # 3 mph, 10 mph, 20 mph
num_path = 10

# Generate Server
users_m0 = []
users_m1 = []
users_m2 = []
idx_counter = 0

for i in range(num_user_m0):
    users_m0 += [User(boundaries, sim_param.time_steps, 0, lamdas, max_speed, num_path)]
    users_m0[-1].generate_MC(servers)
    users_m0[-1].assign_id(idx_counter)
    idx_counter += 1
    
for i in range(num_user_m1):
    users_m1 += [User(boundaries, sim_param.time_steps, 1, lamdas, max_speed, 1)]
    users_m1[-1].generate_MC(servers)
    users_m1[-1].assign_id(idx_counter)
    idx_counter += 1

for i in range(num_user_m2):
    users_m2 += [User(boundaries, sim_param.time_steps, 2, lamdas, max_speed, num_path)]
    users_m2[-1].generate_MC(servers)
    users_m2[-1].assign_id(idx_counter)
    idx_counter += 1

users = users_m0 + users_m1 + users_m2
    
    
"""
Make Jobs
- "I'm just going to do it"
"""

# Job settings
job_type0 = 1 # VR
job_type1 = 0 # Assistant
job_type2 = 0 # AR

jobs0 = []
jobs1 = []
jobs2 = []
idx_counter = 0

total_job_count = job_type0+job_type1+job_type2
draw_job_id = np.random.choice(total_job_count, total_job_count, replace=False)

for i in range(job_type0):
    jobs0 += [Job(job_type = 0,
                  user_id = draw_job_id[idx_counter],
                  time_steps=sim_param.time_steps,
                  job_profiles = job_profiles)]
    idx_counter += 1
    
for i in range(job_type1):
    jobs1 += [Job(job_type = 1,
                  user_id = draw_job_id[idx_counter],
                  time_steps=sim_param.time_steps,
                  job_profiles = job_profiles)]
    idx_counter += 1
    
for i in range(job_type2):
    jobs2 += [Job(job_type = 2,
                  user_id = draw_job_id[idx_counter],
                  time_steps=sim_param.time_steps,
                  job_profiles=job_profiles)]
    idx_counter += 1
    
jobs = jobs0 + jobs1 + jobs2

In [277]:
users[0].server_prob

array([[0. , 0. , 0. , 0. , 0. ],
       [0. , 0. , 0. , 0. , 0. ],
       [1. , 0.9, 0.6, 0.4, 0.6],
       [0. , 0.1, 0.4, 0.6, 0.4],
       [0. , 0. , 0. , 0. , 0. ],
       [0. , 0. , 0. , 0. , 0. ]])

### Make ILP Solver

In [278]:
super_prob = PlanGenerator(users, servers, links, jobs, sim_param)
optim_prob = Optim_PlanGenerator(users, servers, links, jobs, sim_param)

In [279]:
#optim_prob.prob.solve()
print("Status:", constants.LpStatus[optim_prob.prob.status])

Status: Not Solved


### Migration Plan Class

In [280]:
ILP_mig_plan = Migration_Plans(users, jobs, sim_param) 
ILP_mig_plan.from_ILP(optim_prob)

In [281]:
ILP_mig_plan.mig_plan_dict

{0: {'time_slot': array([0., 1., 2., 3., 4.]),
  'user_active_flag': array([1., 1., 1., 1., 1.]),
  'user_voronoi': array([2., 2., 3., 3., 3.]),
  'source_server': array([2., 2., 4., 3., 4.]),
  'dest_server': array([2., 4., 3., 4., 4.]),
  'mig_rate': array([0., 1., 1., 1., 0.]),
  'mig_link_id': array([0., 0., 1., 0., 0.]),
  'service_link_id': array([-1., -1.,  0., -1.,  1.]),
  'service_thruput': array([0.        , 0.        , 0.06312773, 0.        , 0.06312773]),
  'latency': array([    0.        ,     0.        ,  1152.65692936,     0.        ,
         57898.40162042])}}

In [283]:
jobs[0].thruput_req

0.0631277272808401

### Draw Plan From Seq Greedy

In [284]:
SG_prob = SeqGreedy_PlanGenerator(users, servers, links, jobs, sim_param)
SG_plan = Migration_Plans(users,jobs,sim_param)
SG_plan.from_seq_greedy(SG_prob)

usr: 0 reserve: 0


In [285]:
SG_plan.mig_plan_dict[0]

{'time_slot': array([0., 1., 2., 3., 4.]),
 'user_active_flag': array([1., 1., 1., 1., 1.]),
 'user_voronoi': array([2., 2., 3., 3., 3.]),
 'source_server': array([2., 2., 4., 3., 4.]),
 'dest_server': array([2., 4., 3., 4., 4.]),
 'mig_rate': array([0., 1., 1., 1., 0.]),
 'mig_link_id': array([0., 1., 0., 0., 0.]),
 'service_link_id': array([-1., -1.,  0., -1.,  0.]),
 'service_thruput': array([0.        , 0.        , 0.06312773, 0.        , 0.06312773]),
 'latency': array([   0.        ,    0.        , 1152.65692936,    0.        ,
        1152.65692936])}

In [286]:
print(links.distances)

[[  0.           2.52857867   4.09080047   1.52978438   2.98480697
  281.63667495]
 [  2.52857867   0.           2.0726251    1.49596706   1.07830125
  279.75732197]
 [  4.09080047   2.0726251    0.           2.58736949   1.1321021
  277.73509152]
 [  1.52978438   1.49596706   2.58736949   0.           1.52656929
  280.12551313]
 [  2.98480697   1.07830125   1.1321021    1.52656929   0.
  278.85850308]
 [281.63667495 279.75732197 277.73509152 280.12551313 278.85850308
    0.        ]]


In [287]:
2 * False

0